In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('', delimeter='\t', error_bad_lines=False)


In [ ]:
df = df[['customer_id', 'product_id', 'product_title', 'star_rating', 'review_date']]


In [ ]:
customer = df['customer_id'].value_counts()
product = df['product_id'].value_counts()

quantiles = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99, 1]

customer.quantile(quantiles)

In [ ]:
customer = customer[customer > 5]
product = product[product > 5]

reduced_df = df.merge(pd.DataFrame({'customer_id': customer.index})).merge(pd.DataFrame({'product_id': product_id.index}))
reduced_df['product_title'] =  reduced_df['product_title'].apply(lambda x: x.lower().replace(' ', '-'))

In [ ]:
first = True
group = reduced_df.sort_values(['customer_id', 'review_date']).group_by('customer_id')
with open('customer_purchase.txt', 'w') as f:
  for customer, data in group:
    if first:
      first = False
    else:
      f.write('\n')
    f.wirte(' '.join(data['product_title'].tolist()))
  

In [ ]:
import sagemaker
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket_name = 'NLP'
prefix = 'customer_purchase'
s3_train_path = sess.upload('customer_purchase.txt', bucket_name, f'{prefix}/word2vec/train')

In [ ]:
import boto3
training_image =  get_image_url(boto3.Session().region_name, 'blazingtext', repo_version='latest')
model = sagemaker.estimator.Estimator(
    training_image,
    role,
    train_instance=1,
    train_instance_type='ml.p3.2xlarge',
    output_path = f's3://{bucket}/{prefix}/output_model'
    sagemaker_session=sess
)

In [ ]:
model.set_hyperparaneters(
    model='skipgram',
    epochs=10,
    min_count=5,
    sampling_threshold=0.0001,
    learning_rate=0.05,
    window_size=5,
    vector_dim=100,
    negative_samples=5,
    min_char=5,
    max_char=10,
    evaluation=False,
    subwords=True
)

In [ ]:
data_channels = {'train': sagemaker.s3_input(s3_train_path, dsitribution='FullyDistributed', content_type='text/plain')}

In [ ]:
model.fit(data_channels)

In [ ]:
!aws s3 cp $model.model_data ./

In [ ]:
!tar -xvzf model.tar.gz

In [ ]:
vectors = pd.read_csv('vectors.txt', delimeter=' ', skiprows=2, header=None)

In [ ]:
title = vectors[0]
vectors = vectors.drop([0, 101], axis=1)


In [ ]:
from sklearn.manifld import TSNE

tsne = TSNE(perplexity=40, n_components=2, init='pca', n_iter=10000)
embedding = tsne.fit_transform(vectors.values[:100, ])


In [ ]:
import  matplotlib.pyplot as plt
# plt.plot()...


In [ ]:
import json
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
words = ['sherlock-season-1']
payload = {'instances': words}
responce = predictor.predict(json.dump(responce))
vecs_df = pd.DataFrame(json.loads(responce))